# Form Builder

This notebook turns a list of questions and properties,

e.g.

```python
items = [
    {'type' : 'h1', 'text': 'Form Title'},
    {'type' : 'p', 'text': 'Example paragraph text'},
    {'id': 'some-text', 'text': 'Text entry:', 'type': 'text', 'placeholder':'e.g. Blah blah blah'}
]
```

into a properly formatted HTML form, styled using 
[Bootstrap](https://getbootstrap.com/docs/4.0/components/forms/),
which you can copy and paste into your website.

Use it to generate surveys, with minimal time spent messing around editing raw HTML.


In [1]:
from pygments import highlight, lexers
from pygments.lexers.sql import MySqlLexer
from pygments.formatters import HtmlFormatter
from IPython.display import display, HTML, IFrame
from jinja2 import Template

lexer_dict = {
    'html': lexers.HtmlLexer(),
    'jinja': lexers.HtmlDjangoLexer(),
    'sql': MySqlLexer(),
    'php': lexers.templates.HtmlPhpLexer()
}

In [2]:
# Utility functions
def show_code(code: str, language: str = 'jinja'):
    '''Displays a string of code in a given languge, with nice syntax highlighting'''
    formatter = HtmlFormatter(cssclass='pygments')
    lexer = lexer_dict[language]
    html_code = highlight(code, lexer, formatter)
    css = formatter.get_style_defs('.pygments')
    html = '<style>{}</style>{}'.format(css, html_code)
    display(HTML(html))

def render_code(template: str, language: str = 'jinja', **kwargs):
    '''Render a jinja template, using the values provided as kwargs, and print the html output'''
    tpl = Template(template, trim_blocks=True)
    out = tpl.render(**kwargs)
    show_code(out, language)
    
def render_preview(template: str, **kwargs):
    '''Render a jinja template, using the values provided as kwargs, 
    and preview the html output as it will be on the webpage.
    '''
    tpl = Template(template + bootstrap_snippet)
    out = tpl.render(**kwargs)
    display(HTML(out), metadata=dict(isolated=True))
    display(HTML(adjust_snippet))
    
# Some html snippets for previewing forms in this notebook.
bootstrap_snippet = '''
<!-- These are necessary for previewing your form in this notebook -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" crossorigin="anonymous">
<script src="https://code.jquery.com/jquery-3.3.1.slim.min.js" integrity="sha384-q8i/X+965DzO0rT7abK41JStQIAqVgRVzpbzo5smXKp4YfRvH+8abtTE1Pi6jizo" crossorigin="anonymous"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.7/umd/popper.min.js" integrity="sha384-UO2eT0CpHqdSJQ6hJty5KVphtPhzWj9WO1clHTMGa3JDZwrnQq4sF86dIHNDz0W1" crossorigin="anonymous"></script>
<script src="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/js/bootstrap.min.js" integrity="sha384-JjSmVgyd0p3pXB1rRibZUAYoIIy6OrQ6VrjIEaFf/nJGzIxFDsf4x0xIM+B07jRM" crossorigin="anonymous"></script>
'''

adjust_snippet = '''
<script>
// Resize the iframe to the size of it's contents (once loaded)
setTimeout(function(){ 
    let frame = $('iframe')
    let h = frame.contents().find('form').height()
    frame.height(h * 1.1)
}, 1000);
</script>
'''

In [3]:
form_template = '''
<form id="{{ form_id }}">
{% for item in items %}
  <div class="form-group">
  {% if item.type in ['h1', 'h2', 'h3', 'p'] %}
      <{{ item.type }}> {{ item.text }} </{{ item.type }}>
  {% elif item.type in ['select', 'multiselect'] %}
    <label for="{{ item.id }}"> {{ item.text }} </label>
    <select class="form-control" name="{{ item.id }}" {% if item.type=='multiselect' %} multiple {% endif %}>
        {% for opt in item.options %}
        <option value="{{ opt.val }}" {{ opt.special }}> {{ opt.text }} </option>
        {% endfor %}
    </select>
  {% elif item.type == 'textarea' %}
    <label for="{{ item.id }}"> {{ item.text }} </label>
    <textarea class="form-control" 
     id="{{ item.id }}" name="{{ item.id }}"
     placeholder="{{ item.placeholder }}"></textarea>
  {% elif item.type == 'file' %}
    <label for="{{ item.id }}"> {{ item.text }} </label>
    <input type="file" class="form-control-file" id="{{ item.id }}">
  {% elif item.type == 'radio' %}
  <p> {{ item.text }} </p>
  <div class="btn-group" data-toggle="buttons">
      {% for opt in item.options %}
      <label class="btn btn-primary">
        <input type="radio" value="{{ opt.val }}" name="{{ item.id }}" id="{{ item.id }}_{{ loop.index }}"> 
        {{ opt.text }}
      </label>
      {% endfor %}
  </div>
  {% else %}
    <label for="{{ item.id }}"> {{ item.text }} </label>
    <input type="{{ item.type }}" class="form-control" 
     id="{{ item.id }}" name="{{ item.id }}"
     placeholder="{{ item.placeholder }}">
  {% endif %}
  </div>
{% endfor %}

 <button type="submit" class="btn btn-primary" id="button_{{ form_id }}" form_id="{{ form_id }}">
   Submit and Continue
 </button>

</form>
'''

# show_code(form_template) # Uncomment to see template with syntax highlighting

## Generate a form

Edit the list of items below and run the cell to see a preview of your form.
Then, run the cell below that to see the HTML output.

Currently supported item types (the `'type': 'something` bit of the specifications) are:

- `h1`/`h2`,`h3`, `p`, etc: Generate headings or paragraphs of text.
- `text`, `textarea`: For short and long text inputs.
- `number`, `date`, `file`: These should be self-explanatory.
- `radio`: Renders a row of radio buttons (only one can be selected).
- `select`: Select one option from a dropdown.
- `multiselect`: Select multiple options from a dropdown.

More inputs from 
the [HTML forms spec](https://www.w3schools.com/html/html_forms.asp),
and the fancier [Bootstrap forms styles](https://getbootstrap.com/docs/4.0/components/forms/)
will be implemented in the future.
Contributions are welcome.

In [4]:

items = [
    {'type' : 'h1', 'text': 'Form Title'},
    {'type' : 'p', 'text': 'Example paragraph text'},
    {'id': 'some-text', 'text': 'Text entry:', 'type': 'text', 'placeholder':'e.g. Blah blah blah'},
    {'id': 'long-text', 'text': 'Text entry:', 'type': 'textarea', 'placeholder':'e.g. Blah blah blah x10'},
    {'id': 'a-number', 'text': 'Age', 'type': 'number'},
    {'id': 'when', 'text': 'When did it all go wrong?', 'type':'date'}, 
    {'id': 'a-file', 'text': 'Upload something', 'type':'file'}, 
    {'id': 'sex', 'text': 'Sex', 'type':'radio', 
     'options': [
        {'val': 'male',   'text': 'Male'},
        {'val': 'female', 'text': 'Female'},
        {'val': 'other',  'text': 'Other/Prefer not to say'}
    ]},
    {'id': 'select-one', 'text': 'Pick something', 'type':'select', 
     'options': [
         {'val': '', 'special': 'selected disabled', 'text': 'Pick one'}, # Optional default value.
         {'val': 'money',   'text': 'Money'},
         {'val': 'happy', 'text': 'Happiness'},
         {'val': 'other',  'text': 'Other/Prefer not to say'}
    ]},
    {'id': 'select-many', 'text': 'Pick multiple', 'type':'multiselect', 
     'options': [
         {'val': 'money',   'text': 'Money'},
         {'val': 'happy',  'text': 'Happiness'},
         {'val': 'other',  'text': 'Other/Prefer not to say'}
    ]}
]
render_preview(form_template, form_id='my_form', items=items)

In [5]:
render_code(form_template, items=items, form_id='my_form')

## SQL Setup

We can use similar principles to generate SQL code,
which will create a table to store the results from your form.

In [6]:
sql_create_template = '''
CREATE TABLE IF NOT EXISTS {{ table_name }} (
  `id`        integer primary key autoincrement,
  `timestamp` datetime DEFAULT CURRENT_TIMESTAMP,
 {% for var in variables[:-1] %}
 `{{ var }}` varchar,
 {% endfor %}
 `{{ variables[-1] }}` varchar
);
'''

In [7]:
names = [item['id']for item in items  if 'id' in item.keys() ]
render_code(sql_create_template, 'sql', table_name='my_form', variables=names)

## PHP Logging

You might also find this PHP script useful.

In [8]:
php_log_template = '''
<?php

$db = new SQLite3('./data.db');
$inp = file_get_contents('php://input');
$data = json_decode($inp, true);

// Filter out missing values
$data = array_filter($data, function($value) {
  return ($value !== null && $value !== '');
});
// Replace arrays with strings
foreach ($data as $k => $v) {
  if(is_array($v)) {
    $data[$k] = json_encode($v);
  }
}

// Save to database
$keys = '`' . implode('`, `', array_keys($data)) . '`';
$values = "'" . implode("', '", array_values($data)) . "'";
$insert_query = "INSERT INTO responses ($keys) VALUES ($values);";
/* Uncomment to debug */
/* echo $insert_query;*/ 
$res = $db->exec($insert_query);
if(!$res){
  printf("<br><br>\\n\\nError message: %s\\n", mysqli_error($conn));
}
?>
'''

In [9]:
render_code(php_log_template, 'php')